In [1]:
import pandas as pd 
import os
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl


In [ ]:
dict_dfs = {}
others = {}
dfK=pd.DataFrame()

In [ ]:
fs_ticks = 15
fs_axis = 20
fs_title = 35
fs_legend = 20

col_names = {'ti_err':'Type I error',
            'fdr': 'False discovery rate',
            'pwr':'Power',
            'tpr':'True discovery rate',
            'DE':'Power on DE',
            'DM':'Power on DM',
            'DP':'Power on DP','DB':'Power on DB','EE':'EE','EB':'EB'}

path_others = f'/home/data/experimental_data/other_de_methods_scores/'
DE_methods = ["citcdf_perm","MAST","scDD","SigEMD","DESingle","SCDE"]
colors = {'citcdf_asymp':'#E69F00',
          'citcdf_perm':'#E69F00',
          'MAST':'#56B4E9',
          'scDD':'#009E73',
          'SigEMD':'#F0E442',
          'DESingle':'#0072B2',
          'SCDE':'#D55E00',
         }       
    
kfda_directories = [
    'scores_2023_06_ccdf_kfda',
    'scores_2023_06_ccdf_kfda_klin_',
    'scores_2023_06_ccdf_kfda_kgq20_',
    'scores_2023_06_ccdf_kfda_kgq30_',
    'scores_2023_06_ccdf_kfda_kgq40_',
    'scores_2023_06_ccdf_kfda_kgq60_',
    'scores_2023_06_ccdf_kfda_kgq70_',
    'scores_2023_06_ccdf_kfda_kgq80_',]


for directory in kfda_directories:
    if directory not in dict_dfs:
        print(directory)
        path = f'/home/data/experimental_data/{directory}/'
        dfs = pd.DataFrame()
        for file in os.listdir(path):
            df = pd.read_csv(path+file,index_col=0)
            if dfs.empty:
                dfs = df
            else:
                dfs = pd.concat([dfs,df])
        dict_dfs[directory]=dfs

for de in DE_methods:
    for file in os.listdir(path_others):
        if de in file:
            df = pd.read_csv(path_others+file,index_col=0)
            if dfK.empty:
                dfK = df
            else:
                dfK = pd.concat([dfK,df])

In [ ]:
def colorFader(c1,c2,mix=0): 
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

def plot_result_by_n(df,col,label,fig=None,ax=None,ls='-.',color=None,alpha=1,lw=1):
    if fig is None:
        fig,ax = plt.subplots()
    ax.set_ylim(-0.05,1.05)
    df = df[df['n'].isin(ns)]
    df.groupby('n')[col].mean().plot(ax=ax,label=label,ls=ls,c=color,lw=lw,alpha=alpha)
    ax.set_title(col_names[col],fontsize=fs_title)
    ax.set_xticks(df['n'].unique())
    ax.set_xticklabels(df['n'].unique(),fontsize=fs_ticks)
    ax.set_yticks([0,.2,.4,.6,.8,1])
    ylabels = [i._text for i in ax.get_yticklabels()]
    ax.set_yticklabels(ylabels,fontsize=fs_ticks)
    ax.set_xlabel('n',fontsize=fs_axis)
    if col in  ['EE','EB','ti_err','fdr']:
        ax.axhline(.05,c='crimson',ls='--')
        ax.set_yticklabels(ylabels,fontsize=fs_ticks)
    return(fig,ax)

def plot_result_by_t(df,col,label,fig=None,ax=None,ls='-',color=None,alpha=1,lw=1):
    if fig is None:
        fig,ax = plt.subplots()
    ax.set_ylim(-0.05,1.05)
    df.groupby('t')[col].mean().plot(ax=ax,label=label,ls=ls,c=color,lw=lw,alpha=alpha)
    ax.set_title(col_names[col],fontsize=fs_title)
    ax.set_xticks([1,5,10,15,20,25])
    ax.set_xticklabels([1,5,10,15,20,25],fontsize=fs_ticks)
    ax.set_yticks([0,.2,.4,.6,.8,1])
    ylabels = [i._text for i in ax.get_yticklabels()]
    ax.set_yticklabels(ylabels,fontsize=fs_ticks)
    ax.set_xlabel('truncation parameter',fontsize=fs_axis)
    if col in  ['ti_err']:
        ax.axhline(.05,c='crimson',ls='--')
    return(fig,ax)


ignore_DESingle = [12, 17, 33, 42, 51, 73, 93, 116, 124, 150, 172, 204, 234, 235, 237, 240, 273, 282, 
 285, 304, 316, 318, 329, 346, 347, 372, 398, 410, 441, 453, 464, 478, 493]

def plot_others(col,fig=None,ax=None,label=False,lw=2,alpha=1,ls='-'):
    if fig is None:
        fig,ax = plt.subplots()
    for de in DE_methods:
        dfKde = dfK[dfK['m']==de]
        if de in ['DESingle','SigEMD']:
            dfKde = dfKde[~dfKde['s'].isin(ignore_DESingle)]
        if label:
            if de == 'citcdf_perm':
                de_label = 'ccdf'
            else:
                de_label = de
        else:
            de_label = ''
        plot_result_by_n(dfKde,col,label=de_label,fig=fig,ax=ax,ls=ls,color=colors[de],alpha=alpha,lw=lw)

In [ ]:
# Plot main figure

ns = [60,80,100,160,200]
names =[ 'scores_2023_06_ccdf_kfda','scores_2023_06_ccdf_kfda_klin_',]
fig,axes = plt.subplots(ncols=4,nrows=2,figsize=(25,14))    
for col,ax in zip(['ti_err','fdr','pwr','tpr',],axes[0,:]):
    for name in names:
        dfss = dict_dfs[name]
        t = 1 if 'klin' in name else 4
        if 'klin' in name:
            label = 'ktest linear kernel'
            color = '#000000'
        elif 'kgq' in name:
            q = name.split(sep='kgq')[1][:-1]
            label = f'quantile = {q}' 
        else:
            label = 'ktest gauss kernel'
            color = '#CC79A7'
        df = dfss[dfss['m']=='st'][['fdr','tpr','pwr','ti_err','n','t']]
        df = df[df['t']==t]
        plot_result_by_n(df,col,label=label ,lw=5,fig=fig,ax=ax,color=color)
    plot_others(col=col,fig=fig,ax=ax,label=True,lw=3)
    if col == 'ti_err':
        leg = fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0),
        fontsize=25,ncol=8)                        
        leg.get_frame().set_linewidth(0)
for col,ax in zip(['DE','DM','DP','DB'],axes[1,:]):
    for name in names:
        dfss = dict_dfs[name]
        t = 1 if 'klin' in name else 4
        label = 'ktest linear kernel' if 'klin' in name else 'ktest'
        color = '#000000' if 'klin' in name else '#CC79A7'
        df = dfss[dfss['m']=='st'][['DE','DM','DP','DB','EE','EB','n','t']]
        df = df[df['t']==t]
        plot_result_by_n(df,col,label=label,lw=5,fig=fig,ax=ax,color=color)
    plot_others(col=col,fig=fig,ax=ax,lw=3)
fig.tight_layout()
fig.savefig('/home/figures/experimental_data/main.pdf',bbox_inches='tight')


In [ ]:
# Plot supplementary figure 1 
c1='xkcd:dark pink'
c2='xkcd:cerulean'
ns = [60,100,200]
dfss = dict_dfs['scores_2023_06_ccdf_kfda']
lw=2
fig,axes = plt.subplots(ncols=len(cols),nrows=2,figsize=(25,14))    
for col,ax in zip(['ti_err','fdr','pwr','tpr',],axes[0,:]):
            df = dfss[dfss['m']=='st'][cols+['n','t']]
            for i,n in enumerate(ns):
                color = colorFader(c1=c1,c2=c2,mix=i/len(ns))
                dfn = df[df['n']==n]
                plot_result_by_t(dfn,col,label=f'n={n}',lw=lw,fig=fig,ax=ax,color=color)
                if col == 'ti_err' and n==200:
                    leg = fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0),
                    fontsize=20,ncol=8)                        
                    leg.get_frame().set_linewidth(0)
        
for col,ax in zip(['DE','DM','DP','DB'],axes[1,:]):
            df = dfss[dfss['m']=='st'][cols+['n','t']]
            for i,n in enumerate(ns):
                color = colorFader(c1=c1,c2=c2,mix=i/len(ns))
                dfn = df[df['n']==n]
                plot_result_by_t(dfn,col,label=f'n={n}',lw=lw,fig=fig,ax=ax,color=color)
fig.tight_layout()
fig.savefig('/home/figures/experimental_data/supp1.pdf',bbox_inches='tight')

In [ ]:
# Plot supplementary figure 2 (quantiles) 
n=100
names =[ 'scores_2023_06_ccdf_kfda',
'scores_2023_06_ccdf_kfda_kgq20_',
'scores_2023_06_ccdf_kfda_kgq30_',
'scores_2023_06_ccdf_kfda_kgq40_',
'scores_2023_06_ccdf_kfda_kgq60_',
'scores_2023_06_ccdf_kfda_kgq70_',
'scores_2023_06_ccdf_kfda_kgq80_',
]

fig,axes = plt.subplots(ncols=len(cols),nrows=2,figsize=(25,14))    
for col,ax in zip(['ti_err','fdr','pwr','tpr',],axes[0,:]):
    for name,dfss in dict_dfs.items():
        if name in names:
            if 'kgq' in name:
                q = int(name.split(sep='kgq')[1][:-1])
                color = colorFader(c1=c1,c2=c2,mix=q/100)
                label = f'quantile = {q/100}'
            else:
                color='crimson'
                label='median'
            label = label if col == 'ti_err' else None
            df = dfss[dfss['m']=='st'][cols+['n','t']]
            dfn = df[df['n']==n]
            plot_result_by_t(dfn,col,label=label,lw=2,fig=fig,ax=ax,color=color,)
    if col == 'ti_err':
        leg = fig.legend(loc='upper center', bbox_to_anchor=(0.5, 0),
        fontsize=20,ncol=8)                        
        leg.get_frame().set_linewidth(0)
for col,ax in zip(['DE','DM','DP','DB'],axes[1,:]):
    for name,dfss in dict_dfs.items():
        if name in names:
            if 'kgq' in name:
                q = int(name.split(sep='kgq')[1][:-1])
                color = colorFader(c1=c1,c2=c2,mix=q/100)
            else:
                color='crimson'
            df = dfss[dfss['m']=='st'][cols+['n','t']]
            dfn = df[df['n']==n]
            plot_result_by_t(dfn,col,label=None,lw=2,fig=fig,ax=ax,color=color)
fig.tight_layout()
plt.show()
fig.savefig('/home/figures/experimental_data/supp2.pdf',bbox_inches='tight')
